In [30]:
import pandas as pd
from preprocess import preprocess_tweet
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from string import punctuation, digits
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import (
    CountVectorizer, TfidfVectorizer)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import (
    SelectKBest, VarianceThreshold, f_classif)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

## Load data 

In [22]:
n_data = []
p_data = []

# read positive and negative tweet data
# use own path here
with open('/Users/louancillon/Documents/ETHZ/M2/CIL/ETH_CIL22/twitter-datasets/train_neg.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    n_data = lines
print('read negative tweet lines ', len(n_data))

with open('/Users/louancillon/Documents/ETHZ/M2/CIL/ETH_CIL22/twitter-datasets/train_pos.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    p_data = lines
print('read positive tweet lines ', len(p_data))

# fill all tweet data and target labels
tweet_data = p_data + n_data
tweet_labels = [1.0 for _ in p_data] + [0.0 for _ in n_data]
print(len(tweet_data))
print(len(tweet_labels))


read negative tweet lines  100000
read positive tweet lines  100000
200000
200000


In [23]:
df = pd.DataFrame({'review':tweet_data, 'sentiment':tweet_labels})
original = df['review'].map(lambda x: x.split())
df.insert(1, 'original',original)
x_train, x_test,y_train, y_test = train_test_split(df['original'], df['sentiment'], test_size=0.1, random_state = 45)


## Linear baseline 

In [31]:
# Algo
LR = LogisticRegression(
    verbose=0,
    class_weight = "balanced",
    random_state = 0,
    multi_class = "multinomial",
    max_iter=100 ** 2,
)

SVM = svm.SVC(random_state = 0)

RF = RandomForestClassifier(max_depth=2, random_state=0)


In [32]:
def train_eval(x, y, model):
    # Create a pipeline
    baseline = Pipeline([
      ("tf-idf", TfidfVectorizer(lowercase=False, tokenizer=lambda i:i)),
       #TfidfVectorizer(lowercase=False)),
      ("clf", model)
    ])
    
    cv_results = cross_validate(baseline, x_train, y_train, cv=10)
    
    return cv_results
    

In [33]:
LR_eval = train_eval(df['original'], df['sentiment'], LR)
LR_eval

{'fit_time': array([28.92373776, 31.76467896, 32.19085813, 34.60562086, 31.16785312,
        34.63809395, 33.07225609, 34.35304093, 30.51302719, 33.21059513]),
 'score_time': array([0.24328709, 0.35565686, 0.2899971 , 0.2481463 , 0.2603209 ,
        0.25895691, 0.29238486, 0.26996398, 0.25034189, 0.2826519 ]),
 'test_score': array([0.82933333, 0.82888889, 0.83072222, 0.83377778, 0.82777778,
        0.83027778, 0.8265    , 0.83027778, 0.836     , 0.83277778])}

In [ ]:
SVM_eval = train_eval(df['original'], df['sentiment'], SVM)
SVM_eval

In [ ]:
RF_eval = train_eval(df['original'], df['sentiment'], RF)
RF_eval

In [12]:
baseline.fit(X=x_train, 
            y=y_train)
y_pred = baseline.predict(x_test)
acc = accuracy_score(y_test, y_pred)

0.839348

In [20]:
cv_results = cross_validate(baseline, x_train, y_train, cv=10)

KeyboardInterrupt: 

In [ ]:
cv_results

In [11]:
# Create a pipeline
baseline = Pipeline([
  ("tf-idf", TfidfVectorizer(lowercase=False, tokenizer=lambda i:i)),
   #TfidfVectorizer(lowercase=False)),
  ("clf", LR)
])

Pipeline(steps=[('tf-idf',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function <lambda> at 0x12a900bf8>)),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=10000,
                                    multi_class='multinomial',
                                    random_state=0))])